# <center> STAT 301 Project: Proposal </center>

**Group Members (Group 1)** Fiona Wang, Jack Fan, William Suryawidjaja, Ria Ji

## 1. Introduction

Life expectancy is the estimate of the average number of additional years a person is expected to live. Your life expectancy can be affected by various factors such as genetics as well as your lifestyle, which may reduce or increase your life expectancy over time (Passarino, 2016).

Insurance companies typically charge premiums for customers who are likely to pass away shortly after purchasing their policy. This is because they are undertaking a greater risk of paying the entire policy before the customer has invested more than the policy's worth. Therefore, life insurance companies use one's life expectancy to set the pricing of life insurance policies (Social Security, 2019). If there is a huge gap between your age and your life expectancy, there is a lower risk for the insurance company and the policy will be set at a cheaper price.

As such, the motivation behind this group project is to determine the factors that best predicts one's life expectancy. Formally, we are addressing the question: "What are the variables that best predict one's life expectancy?"

To answer this question, we will be using the Life Expectancy Dataset from Kaggle (https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who). The dataset consists of 2938 rows of Life Expectancy data collected from the World Health Organization (WHO) from 2000 to 2015 across 193 countries. Each row has 20 explanatory variable which can be grouped into several broad categories: Immunization related factors, Mortality factors, Economical factors and Social factors(Kumar., 2017).

## 2. Preliminary Results

### 2.1 Data Wrangling

In [ ]:
# install.packages("package_name")
# run this line of code to install the packages if they are not installed on your machine
#install.packages("hardhat")
#install.packages("tidymodels")

# Load required packages
library("GGally")
library("ggplot2")
library("janitor")
library("plyr")
library("tidyverse")
library("reshape")
library("leaps")
library("gridExtra")
#library("tidymodels")

We'll start by importing the dataset, loading it into a dataframe and inspecting the results. The original dataset can be found [here](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who).

In [ ]:
le = read_csv("https://shorturl.at/FV478")

In [ ]:
# In order to display all column names, we need to change the max # of cols displayed
options(repr.matrix.max.cols=22, repr.matrix.max.rows=37)

In [ ]:
# preview the first and last 3 rows of our data
head(le, n = 3)
tail(le, n = 3)

As described in [[3.4 Tidy data, 1]](https://datasciencebook.ca/wrangling.html#tidy-data), a cleaned dataset should have the following attributes:   
> 1. Each row is a single observation,   
> 2. Each column is a single variable, and   
> 3. Each value is a single cell (i.e., its entry in the data frame is not shared with another value).   

Hence, our imported dataset is already clean.

With that being said, we could further improve the usability our data by: 
- Convert variable names into lowercase (for readibility and naming convention), and we will replace space in variable names with underscores for the validity of syntax. We can do these with the help of the janitor library.
- Removing `N/A` values if there exists any. 
- Convert "country" and "status" variables into the `factor` type as they are expected to be. 

In [ ]:
le = clean_names(le)

In [ ]:
le = le %>% 
    mutate(country = as_factor(country)) %>% 
    mutate(status = as_factor(status))

In [ ]:
# remove all NAs

total_rows <- le %>% nrow()

print(sprintf("there are %d rows in the data frame",total_rows))

le <- na.omit(le)

print(sprintf("there are %d rows in the data frame", nrow(le)))

In [ ]:
le = le %>%     
    mutate(country = as_factor(country)) %>%
    mutate(status = as_factor(status))

In [ ]:
# review our data
tail(le, n = 3)

### 2.2 Exploratory Data Analysis: Correlations

In [ ]:
# correlation between numeric variables
cor_matrix <- cor(subset(le, select = -c(country, status) ))
melted <- melt(cor_matrix)

# Make plots larger 
options(repr.plot.width=13, repr.plot.height=11)

ggplot(melted) +
    geom_tile(aes(X1, X2, fill=value), colour = "black") + 
    geom_text(aes(X1, X2, label = round(value,2)), color = "black", size = 3.5) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), 
          axis.text=element_text(size=15, face="bold"), 
          axis.title=element_blank()) +
    scale_fill_gradient2(low = "#6D9EC1", high = "#E46726")+
    guides(fill = guide_colourbar(barwidth = 0.5, barheight = 20))

We noticed some of the variables have high correlation with our response variable, so we decide to explore each of their relationship with life expectancy individually. We plotted linear regression plot for the top 7 variables, which are *gdp, adult_mortality, hiv_aids, income_composition_of_resources, school, percentage_expenditure, and bmi*.  

In [ ]:
options(repr.plot.width=17, repr.plot.height=15, warnings = FALSE)
plot1 <- le %>% ggplot(mapping = aes(gdp, life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs GDP", x="GDP", y="Life Expectancy")

plot2 <- le %>% ggplot(mapping = aes(adult_mortality, life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression for Life Expectancy vs Adult Mortality", x="Adult Mortality", y="Life Expectancy")

plot3 <- le %>% ggplot(mapping=aes(hiv_aids,life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs HIV Aids", x="HIV Aids", y="Life Expectancy")

plot4 <- le %>% ggplot(mapping=aes(income_composition_of_resources,life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs Income Composition of Resources", 
         x="Income Composition of resources", y="Life Expectancy")

plot5 <- le %>% ggplot(mapping=aes(schooling,life_expectancy)) + 
    geom_point(size = 0.5) + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs Schooling", x="Schooling", y="Life Expectancy")

plot6 <- le %>% ggplot(mapping = aes(percentage_expenditure, life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs Expenditure Percentage", x="Expenditure Percentage", y="Life Expectancy")

plot7 <- le %>% ggplot(mapping=aes(bmi,life_expectancy)) + 
    geom_point() + 
    geom_smooth(method="lm", se = FALSE) + 
    theme(text=element_text(size=15, face="bold"))+
    labs(title = "Linear regression plot for Life Expectancy vs BMI", x="BMI", y="Life Expectancy")

grid.arrange(plot1, plot2, plot3, plot4, plot5, plot6, plot7, nrow = 4)

From the graph above, gdp, hiv_aids, and percentage_expenditure seems to be mostly clustered along the lower end of the x-axis, with few points on the larger scale. For other four variables, points are mostly scattered apart along the regression line. We noticed that adult mortality have a strong linear association with life expectancy, but we are excluding this variable in later analysis since it is very similar to our response variable, both are measure of people's death, a variation of life expectancy, its strong correlation might effect the exploration on other possible predictor variables. For the same reason, we are also excluding infant_deaths and under_five_deaths, it is just a quantitative measurement of people with lower life expectancy. The country variable is not included as well since we are not considering life expectancy difference across countries and it cannot classify as a category variable.  

### 2.3 Exploratory Data Analysis: Visualizations

First, we will start with exploring the distribution of life expectancy using a histogram. We can see from the graph below, the distribution is unimodel but not normal and it is slightly left-skewed. The mean year of life expectancy is slgihtly below 70, but there are above 200 people has life expectancy of 73 to 74 years. The graph below gives us a general view of the distribution of our response variable. 

In [ ]:
options(repr.plot.width=9, repr.plot.height=7)
le %>% ggplot(aes(x = life_expectancy)) + 
    geom_histogram(alpha = 2,color = "black", fill = "#6d9ec1") +
    geom_vline(aes(xintercept=mean(life_expectancy)), color="blue",
             linetype="dashed", lwd = 2)+
    labs(title="Life Expectancy Histogram Plot",x="Life Expectancy (years)", y = "Count")+
    theme(text=element_text(size=15, face = "bold"))

Also using a Q-Q plot to check if this data set is a normal distribution, which is one of the assumptions for doing linear regression. The graph below shows that our data set is not very normally distributed, it is a bit tailed and left skew. 

In [ ]:
qqnorm(le$life_expectancy)
qqline(le$life_expectancy, col = "steelblue", lwd = 3)

Then we are exploring the relationship between Life Expectancy over the years from 2000 to 2015 according to country's developing status. From the graph below, the two lines of mean for both status showing an increasing trend, which means people have higher life expectancy on average in 2015 than 2000. We can also see a clear difference between developing countries and developed countires points, most of the data points are above 70 for developed country, while developing coutries' data points are mostly below 80. So considering this difference, we might need to include interaction terms in later analysis. 

In [ ]:
options(repr.plot.width=9, repr.plot.height=7)
le %>% ggplot(mapping=aes(year,life_expectancy, color=status)) + 
    geom_point() + 
    geom_smooth(method="lm",se=FALSE) + 
    theme(plot.title=element_text(size=17, face = "bold"), 
        axis.title=element_text(size=15, face="bold"), 
        legend.title = element_text(size = 15, face = "bold")) + 
    labs(title="Life Expectancy from 2000-2015", x = "Year", y = "Life Expectancy") + 
    geom_jitter()

## 3. Methods

- What methods do you plan on using?

We will perform EDA to detect heteroscedasticity and remove multicollinearity. Afterwards, we will split the model into the training set and the testing set with a 75-25 split.

Using the training set, we will perform a forward predictive modelling to and select our model depending on the model with the best Mallow's Cp/ BIC value. Using the same training set, we will create a linear model using the selected input variables in an additive model.

Finally, we will evaluate the performance of the linear model using the testing set. 

- What do you expect to achieve?

Through our analysis, we expect to see a strong relationship between the selected input variables that predict one’s life expectancy the best. We will be judging the relationship, as well as how good our model is, by the test RMSE that we obtain.

- What impact could your results have?

We hope that our results can be used by life insurance companies to fine tune their existing models, and help them better adjust the price of insurance plans depending on one’s expected life expectancy.



## 4. References


1. Passarino, Giuseppe, et al. “Human Longevity: Genetics or Lifestyle? It Takes Two to Tango.” Immunity &amp; Ageing : I &amp; A, BioMed Central, 5 Apr. 2016, https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4822264/.
2. “Social Security.” Actuarial Life Table, https://www.ssa.gov/oact/STATS/table4c6.html. 
3. KumarRajarshi. “Life Expectancy (WHO).” Kaggle, 10 Feb. 2018, https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who. 
4. Tiffany Timbers, Trevor Campbell. “Data Science.” Data Science, 24 Sept. 2022, https://datasciencebook.ca/. 
